In [5]:
import torch
from torch import nn, optim
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

In [8]:
ids = pd.read_csv("data/IdLookupTable.csv", index_col = False)
ids.head(10)

,RowId,ImageId,FeatureName,Location
0,1,1,left_eye_center_x,NaN
1,2,1,left_eye_center_y,NaN
2,3,1,right_eye_center_x,NaN
3,4,1,right_eye_center_y,NaN
4,5,1,left_eye_inner_corner_x,NaN
5,6,1,left_eye_inner_corner_y,NaN
6,7,1,left_eye_outer_corner_x,NaN
7,8,1,left_eye_outer_corner_y,NaN
8,9,1,right_eye_inner_corner_x,NaN
9,10,1,right_eye_inner_corner_y,NaN


In [12]:
train_df = pd.read_csv("data/training.csv", index_col = False)
train_df.head(3)

,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,right_eye_inner_corner_y,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,66.033564,39.002274,30.227008,36.421678,59.582075,39.647423,73.130346,39.969997,36.356571,37.389402,...,57.066803,61.195308,79.970165,28.614496,77.388992,43.312602,72.935459,43.130707,84.485774,238 236 237 238 240 240 239 241 241 243 240 23...
1,64.332936,34.970077,29.949277,33.448715,58.856170,35.274349,70.722723,36.187166,36.034723,34.361532,...,55.660936,56.421447,76.352000,35.122383,76.047660,46.684596,70.266553,45.467915,85.480170,219 215 204 196 204 211 212 200 180 168 178 19...
2,65.057053,34.909642,30.903789,34.909642,59.412000,36.320968,70.984421,36.320968,37.678105,36.320968,...,53.538947,60.822947,73.014316,33.726316,72.732000,47.274947,70.191789,47.274947,78.659368,144 142 159 180 188 188 184 180 167 132 84 59 ...


In [13]:
train_df.dtypes

left_eye_center_x            float64
left_eye_center_y            float64
right_eye_center_x           float64
right_eye_center_y           float64
left_eye_inner_corner_x      float64
left_eye_inner_corner_y      float64
left_eye_outer_corner_x      float64
left_eye_outer_corner_y      float64
right_eye_inner_corner_x     float64
right_eye_inner_corner_y     float64
right_eye_outer_corner_x     float64
right_eye_outer_corner_y     float64
left_eyebrow_inner_end_x     float64
left_eyebrow_inner_end_y     float64
left_eyebrow_outer_end_x     float64
left_eyebrow_outer_end_y     float64
right_eyebrow_inner_end_x    float64
right_eyebrow_inner_end_y    float64
right_eyebrow_outer_end_x    float64
right_eyebrow_outer_end_y    float64
nose_tip_x                   float64
nose_tip_y                   float64
mouth_left_corner_x          float64
mouth_left_corner_y          float64
mouth_right_corner_x         float64
mouth_right_corner_y         float64
mouth_center_top_lip_x       float64
m

In [17]:
train_df.isna().sum()

left_eye_center_x              10
left_eye_center_y              10
right_eye_center_x             13
right_eye_center_y             13
left_eye_inner_corner_x      4778
left_eye_inner_corner_y      4778
left_eye_outer_corner_x      4782
left_eye_outer_corner_y      4782
right_eye_inner_corner_x     4781
right_eye_inner_corner_y     4781
right_eye_outer_corner_x     4781
right_eye_outer_corner_y     4781
left_eyebrow_inner_end_x     4779
left_eyebrow_inner_end_y     4779
left_eyebrow_outer_end_x     4824
left_eyebrow_outer_end_y     4824
right_eyebrow_inner_end_x    4779
right_eyebrow_inner_end_y    4779
right_eyebrow_outer_end_x    4813
right_eyebrow_outer_end_y    4813
nose_tip_x                      0
nose_tip_y                      0
mouth_left_corner_x          4780
mouth_left_corner_y          4780
mouth_right_corner_x         4779
mouth_right_corner_y         4779
mouth_center_top_lip_x       4774
mouth_center_top_lip_y       4774
mouth_center_bottom_lip_x      33
mouth_center_b

In [ ]:
print(len(train_df['Image'][0]))
print(train_df['Image'][0])

In [29]:
train_df.shape

(7049, 31)

In [31]:
train_15_landmarks = train_df[~(train_df.isnull().sum(axis=1) > 20)].copy()
train_4_landmarks = train_df[(train_df.isnull().sum(axis=1) > 20)].copy()

In [34]:
train_15_landmarks.shape, train_4_landmarks.shape

((2284, 31), (4765, 31))

In [37]:
train_15_landmarks.iloc[0:100, :].to_csv("data/val_15.csv")
train_15_landmarks.iloc[100:, :].to_csv("data/train_15.csv")

In [38]:
train_4_landmarks.iloc[0:200, :].to_csv("data/val_4.csv")
train_4_landmarks.iloc[200:, :].to_csv("data/train_4.csv")

In [77]:
train_4 = pd.read_csv("data/train_15.csv", index_col = False)
train_4.head(1)

,Unnamed: 0,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,100,64.540235,33.425647,31.731529,35.949318,59.492471,34.372024,69.902824,33.110188,35.832706,...,58.662776,68.325882,69.703765,34.886118,73.173882,50.659765,67.180235,50.659765,79.168235,254 251 251 252 236 181 106 67 57 55 73 83 74 ...


In [59]:
import numpy as np
np.array(train_4.iloc[1,30].split(), dtype=np.float32)

AttributeError: 'numpy.float64' object has no attribute 'split'

In [ ]:
train_4.iloc[2,31]

In [78]:
train_4.head(1)

,Unnamed: 0,left_eye_center_x,left_eye_center_y,right_eye_center_x,right_eye_center_y,left_eye_inner_corner_x,left_eye_inner_corner_y,left_eye_outer_corner_x,left_eye_outer_corner_y,right_eye_inner_corner_x,...,nose_tip_y,mouth_left_corner_x,mouth_left_corner_y,mouth_right_corner_x,mouth_right_corner_y,mouth_center_top_lip_x,mouth_center_top_lip_y,mouth_center_bottom_lip_x,mouth_center_bottom_lip_y,Image
0,100,64.540235,33.425647,31.731529,35.949318,59.492471,34.372024,69.902824,33.110188,35.832706,...,58.662776,68.325882,69.703765,34.886118,73.173882,50.659765,67.180235,50.659765,79.168235,254 251 251 252 236 181 106 67 57 55 73 83 74 ...


In [ ]:
train_4.drop(columns="Unnamed: 0")

Installing EfficientNet

In [68]:
import sys
sys.path.append('efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master')
from efficientnet_pytorch import EfficientNet


In [69]:
EfficientNet

efficientnet_pytorch.model.EfficientNet

In [70]:
sys.path

['/Users/ayushgupta/Ayush/GitHub/Facial Keypoint Detection',
 '/Users/ayushgupta/anaconda3/lib/python310.zip',
 '/Users/ayushgupta/anaconda3/lib/python3.10',
 '/Users/ayushgupta/anaconda3/lib/python3.10/lib-dynload',
 '',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-macosx-11.1-arm64.egg',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/aeosa',
 '/Users/ayushgupta/Ayush/Data Science/Radiology',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/mpmath-1.2.1-py3.10.egg',
 '/Users/ayushgupta/anaconda3/lib/python3.10/site-packages/pycurl-7.45.1-py3.10-macosx-11.1-arm64.egg',
 '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
 '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
 'efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master']

In [ ]:
#from torchsummary import summary

model = EfficientNet.from_pretrained("efficientnet-b0")
model._fc = nn.Linear(1280, 30)
model